Домашнє завдання 
==

В цьому завданні потрібно побудувати рекомендатор, використовуючи колаборативну фільтрацію. На вибір можна обрати наступні алгоритми із бібліотеки surprise
* KNNWithMeans - алгоритм, що враховує середні значення
* KNNWithZScore - алгоритм, що використовує нормалізацію
* SVD

Датасет - Book crossing. 

Фінальна функція приймає на вхід користувача (User-Id) та видає йому 10 рекомендацій книг. 

За бажанням можна також приділити більше уваги очищенню даних
* Прибрати дублікати
* Видалити схожі, але різні назви однієї і тієї ж книги

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from surprise import Dataset
from surprise import Reader
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV

from warnings import filterwarnings
filterwarnings("ignore")

Згідно з <b>BookCrossingEDA</b> файл users_ratings вже містить юзерів та книги, які мають достатню кількість відгуків (приймаємо за мінімум вказане значення у 50 відгуків)

Зчитаємо дані та пофіксимо помилки в колонці Age. Евристично припускаємо, що користувачі з віком > 120 або <4 вказали вік помилково. 

Створюємо вікові групи відповідно до віку користувача.

In [301]:
df = pd.read_csv("datasets/book-crossing/users-ratings.csv")
books = pd.read_csv("datasets/book-crossing/Books.csv", on_bad_lines="skip", delimiter=";")

df["Age"] = pd.to_numeric(df["Age"], errors="coerce")
df["Age"] = df["Age"].apply(lambda x: x if x >= 4 and x <= 120 else np.nan)

bins = [4, 14, 21, 60, 120] 
df["Age-bins"] = pd.cut(df["Age"], bins)

df.tail()

,User-ID,Age,ISBN,Rating,Age-bins
127308,262070,28.0,0156027321,0,"(21, 60]"
127309,262070,28.0,0375703861,0,"(21, 60]"
127310,262070,28.0,0439064864,0,"(21, 60]"
127311,262070,28.0,0439139600,0,"(21, 60]"
127312,262070,28.0,0971880107,0,"(21, 60]"


За умовами, які були проговорені на заняттях, якщо рейтинг = 0, то це означає, що юзер не дав рейтинг відповідній книзі.

Підготуємо дані для подальшого прогнозування. Для тренування моделі використовуватимемо лише юзерів з рейтингом > 0.

In [80]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df[["User-ID", "ISBN", "Rating"]][(df["Rating"] > 0)], reader)

Проведемо крос-валідацію 3 різних алгоритмів KNN, щоб обрати кращий

In [81]:
algos = [
    {"name":"KNN basics", "func": KNNBasic},
    {"name":"KNN with Z-score", "func": KNNWithZScore},
    {"name": "KNN with means", "func": KNNWithMeans}
]

sim_options = {"name": "cosine", "user_based": False}

results = []
for algo in algos:
    algorithm = algo.get("func")
    name = algo.get("name")
    result = cross_validate(algorithm(sim_options=sim_options), data, measures=['RMSE'], cv=3, verbose=False)
    results.append({"algorithm": name, "rmse": np.mean(result.get("test_rmse"))})

results_df = pd.DataFrame(results)
results_df.sort_values(by="rmse")

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


,algorithm,rmse
2,KNN with means,1.673851
1,KNN with Z-score,1.692294
0,KNN basics,1.712100


KNN with means дає найкращий показник rmse.

Створимо train / test-датасет та застосуємо до них обрану модель.

In [83]:
train, test = train_test_split(data)

In [103]:
algorithm = KNNWithMeans(sim_options=sim_options)
algorithm.fit(train)
prediction = algorithm.predict(243, "0060977493")
prediction.est

Computing the cosine similarity matrix...
Done computing similarity matrix.


7.016351234726175

На прикладі першого взятого юзера бачимо, що алгоритм дав est 7.01 проти 7 rui. Застосуємо натреновану модель до тест-датасету та оцінимо результати.  

In [104]:
predictions = algorithm.test(test, verbose=True)

user: 129074     item: 0515095826 r_ui = 7.00   est = 5.83   {'actual_k': 10, 'was_impossible': False}
user: 11676      item: 0446391301 r_ui = 7.00   est = 8.86   {'actual_k': 40, 'was_impossible': False}
user: 252695     item: 0345378482 r_ui = 9.00   est = 7.74   {'actual_k': 13, 'was_impossible': False}
user: 170415     item: 0060987103 r_ui = 7.00   est = 7.50   {'actual_k': 3, 'was_impossible': False}
user: 127359     item: 0373250126 r_ui = 7.00   est = 6.77   {'actual_k': 7, 'was_impossible': False}
user: 66680      item: 0425168298 r_ui = 7.00   est = 7.28   {'actual_k': 16, 'was_impossible': False}
user: 31826      item: 0345386108 r_ui = 10.00   est = 9.94   {'actual_k': 40, 'was_impossible': False}
user: 87746      item: 0679745203 r_ui = 5.00   est = 7.90   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 226965     item: 0345422406 r_ui = 10.00   est = 8.52   {'actual_k': 14, 'was_impossible': False}
user: 49460      item: 0312421273 r_ui = 10.00  

Введемо метрику hit_counts для оцінки результатів

In [179]:
like = 7

hits = [1 for prediction in predictions if prediction.est >= like and prediction.r_ui >= like]
hit_rate = sum(hits) / len(predictions)

hit_rate

0.6941346280015745

Впровадимо функцію для отримання топ-10 рекомендацій для конкретного юзера.

В функцію додамо пост-обробку, а саме: не рекомендувати книги, які юзер вже оцінив.

Також додаємо можливість вивести загальний топ-список книг для "холодного старту".

In [302]:
def get_recommendations(user_id: int, top_n: int = 10):
    unique_users = df["User-ID"].unique().tolist()
    if user_id in unique_users:
        algorithm = KNNWithMeans(sim_options=sim_options)
        data = Dataset.load_from_df(df[["User-ID", "ISBN", "Rating"]][(df["Rating"] > 0)], reader)
        model = algorithm.fit(data.build_full_trainset()) # Ось тут не впевнений чи правильно. Коли ми вже обрали модель, то на проді ми ж навчаємо її на всіх наявних даних, вірно?
        not_rated_by_user = df[(df["Rating"] == 0) & (df["User-ID"] == user_id)]["ISBN"].unique().tolist()
        predictions = [model.predict(user_id, book_id) for book_id in not_rated_by_user]
        predictions.sort(key=lambda x: x.est, reverse=True)
        top_n = predictions[:top_n]
        result = pd.DataFrame([{"ISBN": row.iid, "Predicted rating": row.est} for row in top_n])
        return result.merge(right=books, how="left", on="ISBN")
    else:
        grouped_df = df[(df["Rating"]>0)].groupby("ISBN", as_index=False).agg({"User-ID": "count", "Rating": "mean"})
        grouped_df.sort_values(by=["User-ID", "Rating"], ascending=False, inplace=True, ignore_index=True)
        return grouped_df[(grouped_df["Rating"] >= like)][["ISBN", "Rating"]][:top_n]

Протестуємо результат на реальному юзері.

In [305]:
recommendation = get_recommendations(11676)

user_11676 = df[(df["User-ID"] == 11676)]
real_ratings = user_11676[(user_11676["ISBN"].isin(recommendation["ISBN"].unique().tolist()))]

result = pd.merge(left=recommendation[["ISBN", "Predicted rating", "Title"]], right=real_ratings[["ISBN", "Rating"]], how="inner", on="ISBN")
result

Computing the cosine similarity matrix...
Done computing similarity matrix.


,ISBN,Predicted rating,Title,Rating
0,3257229534,10.000000,Der Vorleser,0
1,3426029553,9.813555,NaN,0
2,0345339711,9.434301,"The Two Towers (The Lord of the Rings, Part 2)",0
3,0439136350,9.289214,Harry Potter and the Prisoner of Azkaban (Book 3),0
4,043935806X,9.092557,Harry Potter and the Order of the Phoenix (Boo...,0
5,0446310786,9.087503,To Kill a Mockingbird,0
6,0142001740,9.049839,The Secret Life of Bees,0
7,0440217563,9.049294,Voyager,0
8,0552137030,9.031515,NaN,0
9,0385503857,9.013277,Oryx and Crake,0


Переконуємося, що все працює і ми рекомендуємо книги, які юзер до цього не оцінював.

Переконаємося, що це схоже на те, що юзер лайкав в реальності.

In [312]:
real_rated = df[(df["User-ID"] == 11676)].merge(right=books, how="left", on="ISBN")
real_rated[["User-ID", "ISBN", "Rating", "Title"]].sort_values(by="Rating", ascending=False)[:10]

,User-ID,ISBN,Rating,Title
729,11676,0440967694,10,The Outsiders
997,11676,0515136530,10,"Key of Valor (Roberts, Nora. Key Trilogy, 3.)"
208,11676,0316693324,10,When the Wind Blows
209,11676,0316734837,10,Fortune's Rocks : A Novel
1091,11676,0553574574,10,Beach Music
213,11676,0316777722,10,Me Talk Pretty One Day
217,11676,0316779490,10,Naked
1063,11676,0553375407,10,Ishmael: An Adventure of the Mind and Spirit
1062,11676,0553348973,10,Still Life with Woodpecker
1061,11676,0553297260,10,Darkness


Бачимо, що юзер лайкав новели і ми рекомендуємо йому також новели + мейнстрім типу Гарі Потера. Можна було б додатково створити стоп-лист з книг, щоб не рекомендувати мейнстрім, але в цьому завданні пропустимо цей момент.

Протестуємо "холодний старт" на невідомому для моделі юзері.

In [313]:
unknown_user = get_recommendations(2)

books = pd.read_csv("datasets/book-crossing/Books.csv", on_bad_lines="skip", delimiter=";")
unknown_user.merge(books[["Title", "ISBN"]], how="left", on="ISBN")

,ISBN,Rating,Title
0,0316666343,8.262443,The Lovely Bones: A Novel
1,0385504209,8.644444,The Da Vinci Code
2,0312195516,8.457364,The Red Tent (Bestselling Backlist)
3,059035342X,9.081301,Harry Potter and the Sorcerer's Stone (Harry P...
4,0142001740,8.745763,The Secret Life of Bees
5,043935806X,8.982301,Harry Potter and the Order of the Phoenix (Boo...
6,0679781587,8.672566,NaN
7,0345337662,7.693694,Interview with the Vampire
8,044021145X,7.907407,The Firm
9,0446672211,8.396226,Where the Heart Is (Oprah's Book Club (Paperba...


Виводиться мейнстрім, як і треба для невідомого юзера.

# Запитання

1. Не розумію як тепер правильно додати до KNN фічу Age. Чи це повинно впроваджуватися на етапі семплінгу?
2. Не зовсім розумію логіку фільтрації від 50 відгуків по книзі і по юзеру на етапі пре-процесингу? Це через те, що ми не хочемо рекомендувати книги, які отримали мало оцінок і не хочемо показувати рекомендації юзерам, які поки що залишили мало відгуків, вірно? 
3. Чому було обрано саме 50 відгуків? Як обрати оптимальну кількість відгуків для потрапляння юзера в модель? 